In [2]:
import numpy as np

In [49]:
from numpy.matrixlib.defmatrix import N
import sys
np.set_printoptions(threshold = sys.maxsize)
def make_try_set(l,n,metrix,check,size):
  
  trys = [[0 for i in range(l+1)]]
  results = []
  while len(trys) != 0:
    t = trys.pop(0)
    
    if np.sum(np.dot(metrix,np.array(t)[:l])%4 - check) == 0:
      result = [0 for i in range(size-l)]
      result.extend(t[:l])
      results.append(np.array(result))
      
      
    
    iter =  t[-1] 
    for i in range(l-iter):
      n_t = t.copy()
      n_t[-1] = i+iter
      if n_t[iter+i] < n-1:
        n_t[iter+i]+=1
        trys.append(n_t)
  
  return results

def make_basises_metrix(row, col):
  move = np.array([[-1,0],[1,0],[0,-1],[0,1],[0,0]])
  basises = []
  for i in range(row):
    for j in range(col):
      points = np.array([i,j]) + move
      basis = np.zeros(row*col)
      for point in points:
        if point[0] < 0 or point[1] <0 or point[0] >= row or point[1] >= col:
          pass
        else:
          basis[row*point[0] + point[1]] = 1;
      basises.append(basis)
  return np.array(basises).T

def G4_eliminate(src,tar,c):
  if src[c] == 0 or tar[c] == 0:
    return tar.copy()

  if src[c] == 1:
    if tar[c] == 1:
      return (3*src.copy()+tar.copy())%4
    elif tar[c] == 2:
      return (2*src.copy()+tar.copy())%4
    else:
      return (src.copy()+tar.copy())%4
  elif src[c] == 2:
    if tar[c] == 1:
      return (src.copy()+2*tar.copy())%4
    elif tar[c] == 2:
      return (src.copy()+tar.copy())%4  
    else:
      return (src.copy()+2*tar.copy())%4
  else:
    if tar[c] == 1:
      return (src.copy()+tar.copy())%4
    elif tar[c] == 2:
      return (2*src.copy()+tar.copy())%4
    else:
      return (3*src.copy()+tar.copy())%4  



def swap_row(src,dst,metrix):
  tem = metrix[src].copy()
  metrix[src] = metrix[dst].copy()
  metrix[dst] = tem

def change_pivot(pivot_row,pivot_col,metrix):
  r,c = metrix.shape
  result = -1
  for j in range(pivot_col,c):
    for i in range(pivot_row,r):
      if metrix[i][j] != 0:
        swap_row(pivot_row,i,metrix)
        result = j
        return result

def G4_gauss_elimination(metrix,r,c):
  position = r
  initial_metrix = metrix.copy()
  for j in range(c):
    if metrix[j][j] == 0:
      state = change_pivot(j,j,metrix)
      if(state == -1):
        return
    else:
      state = j    
    cur_row = metrix[j].copy()
    if cur_row[j] == 0:
      position = j
      break 
    for i in range(j+1,r):
      if metrix[i][j] != 0:
        result = G4_eliminate(cur_row,metrix[i],state)
        metrix[i] = result.copy() 

  if position > c:
    position = c

  nr = position
  nc = position
  init_result_set = make_try_set(c-nc+1,4,metrix[nc-1,nc-1:c],metrix[nc-1][c],c)
  
  for w in range(nc-1):
    sub_solutions = []
    for s in init_result_set:
      v = 0
      value = np.dot(metrix[:,:c][nc-2-w],s)
      while (metrix[nc-2-w][nc-2-w]*v+value)%4 != metrix[nc-2-w][c]:
        v+=1
      s[nc-2-w] = v
      if metrix[nc-2-w][nc-2-w] == 2:
        n_s = s.copy()
        n_s[nc-2-w] = v+2;
        sub_solutions.append(n_s)
    init_result_set.extend(sub_solutions)
  
  candidate = []
  real = np.array([3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3])
  min = 1000000000
  for s in init_result_set:
    middle = np.dot(initial_metrix[:,:c],s)%4
    if int(np.sum(abs(initial_metrix[:,c] - middle))) == 0:
        num = int(np.sum(s)) 
        if  num < min:
          min = num
          candidate = s

  return int(np.sum(candidate))

def make_try_set2(clockHands):
  init_condition = np.array(clockHands)
  h ,w = init_condition.shape
  init_vector = init_condition.flatten()
  basises_metrix = make_basises_metrix(h,w)
  inv = np.linalg.pinv(basises_metrix)
  det = int(round(np.linalg.det(basises_metrix)))
  i = 0
  
  while (4*i+1)%abs(det) != 0:
    i+=1
  alpha = 4*i+1
      
  result = np.dot(inv,-1*alpha*init_vector)
  result = np.array([int(round(i)) for i in result])
  
  return int(round(np.sum(result%4)))

def solution(clockHands):
  metrix = np.array(clockHands)
  r,c = metrix.shape
  basises_metrix = make_basises_metrix(r,c)

  b_r, b_c = basises_metrix.shape
  init_vector = metrix.flatten()

  negative = np.ones(len(init_vector))*4
  gauss_metrix = np.c_[basises_metrix,(negative - init_vector)%4]
  
  if r == 5 or r == 4:
    return G4_gauss_elimination(gauss_metrix,b_r,b_c)
  else:
    return make_try_set2(clockHands)





import time
test = [[0,3,3,0],[3,2,2,3],[0,3,2,0],[0,3,3,3]]
test2 = [[0,0,1],[0,0,1],[0,1,1]]
test3 = [[1,2],[3,0]]
test4=[[1,1,0],[1,0,0],[1,0,0]]
test5 = [[1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
test6 = [[1,1,0,1,1],[1,0,0,0,1],[0,0,0,0,0],[0,0,0,0,1],[0,0,0,1,1]]
test7 = [[1,1,0,0,0],[1,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]

start = time.time()
print(solution(test6))
print("run time",time.time()-start)




[3 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]
9
run time 5.011192798614502


In [26]:
def make_basises_metrix(row, col):
  move = np.array([[-1,0],[1,0],[0,-1],[0,1],[0,0]])
  basises = []
  for i in range(row):
    for j in range(col):
      points = np.array([i,j]) + move
      basis = np.zeros(row*col)
      for point in points:
        if point[0] < 0 or point[1] <0 or point[0] >= row or point[1] >= col:
          pass
        else:
          basis[row*point[0] + point[1]] = 1;
      basises.append(basis)
  return np.array(basises).T

def make_try_set1(l,n,metrix,check):
  trys = [[0 for i in range(l+1)]]
  results = []
  while len(trys) != 0:
    t = trys.pop(0)
    print(t)
    if np.sum(np.dot(metrix,np.array(t)[:l]) - check) == 0:
      print(np.dot(metrix,np.array(t)[:l]))
      print(check)
      print(np.array(t)[:l])
      return int(np.sum(np.array(t)[:l]))
    
    iter =  t[-1] 
    for i in range(l-iter):
      n_t = t.copy()
      n_t[-1] = i+iter
      if n_t[iter+i] < n-1:
        n_t[iter+i]+=1
        trys.append(n_t)
  
  return None

def make_try_set2(clockHands):
  init_condition = np.array(clockHands)
  h ,w = init_condition.shape
  init_vector = init_condition.flatten()
  basises_metrix = make_basises_metrix(h,w)
  inv = np.linalg.pinv(basises_metrix)
  det = int(round(np.linalg.det(basises_metrix)))
  i = 0
  
  while (4*i+1)%abs(det) != 0:
    i+=1
  alpha = 4*i+1
      
  result = np.dot(inv,-1*alpha*init_vector)
  result = np.array([int(round(i)) for i in result])
  
  return int(round(np.sum(result%4)))

def solution2(clockHands):
  metrix = np.array(clockHands)
  r,c = metrix.shape
  basises_metrix = make_basises_metrix(r,c)

  b_r, b_c = basises_metrix.shape
  init_vector = metrix.flatten()
  
  if r == 5 or r == 4:
    return make_try_set1(b_r,4,basises_metrix,(np.ones(b_r)*4 - init_vector)%4)
  else:
    return make_try_set2(clockHands)

test = [[0,3,3,0],[3,2,2,3],[0,3,2,0],[0,3,3,3]]
test2 = [[0,0,1],[0,0,1],[0,1,1]]
test3 = [[0,0],[0,1]]
test4 = [[0,0,0,0,0,0,1,1],[0,0,0,0,0,0,0,1],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]]
test5 = [[1,1,0,0,0],[1,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
print(solution2(test5))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 11]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 12]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 13]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
basis = make_basises_metrix(4,4)
initialv = np.array(test).flatten()
invers = np.linalg.pinv(basis)
result = np.dot(invers,-1*initialv)
print(-1*initialv)
result2 = [int(round(i)) for i in np.dot(basis,result)]
print(result2)
print(result)

[ 0 -3 -3  0 -3 -2 -2 -3  0 -3 -2  0  0 -3 -3 -3]
[0, -3, -3, 0, -3, -2, -2, -3, 0, -3, -2, 0, 0, -3, -3, -3]
[-3.15  1.65  1.35 -3.85  1.9  -3.05 -1.95  2.1   1.1  -0.95 -0.05  0.9
 -1.85  0.35 -0.35 -3.15]


In [26]:
print(np.sum(np.array([1,2,3,4]) == np.array([1,2,3,4])))

4


In [ ]:

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
from itertools import product
def solution(clockHands):
    answer = int(1e9)
    n = len(clockHands)

    def calc(x, y):
        dx = [0,0,0,1,-1]
        dy = [0,1,-1,0,0]
        temp = 0
        for k in range(5):
            nx = x + dx[k]
            ny = y + dy[k]
            if not (0 <= nx < n and 0 <= ny < n):
                continue
            temp += data[nx][ny]
        return (clockHands[x][y] + temp) % 4

    for li in product(list(range(4)), repeat=n):
        #print(li)
        data = [[0] * n for _ in range(n)]
        data[0] = list(li).copy()
        for i in range(1, n):
            for j in range(n):
                data[i][j] = (4 - calc(i-1, j)) % 4
        for j in range(n):
            if calc(n-1, j) != 0:
                break
        else:
            temp = sum([sum(t) for t in data])
            answer = min(answer, temp)

    return answer

In [10]:
a = ["banana", "apple", "rice", "pork", "pot"] 
b = [3, 2, 2, 2, 1] 
c = ["chicken", "apple", "apple", "banana", "rice", "apple", "pork", "banana", "pork", "rice", "pot", "banana", "apple", "banana"]
def solution(want, number, discount):
    bucket = {}
    my_list = {}
    total = 0
    state = True
    for bind in zip(want,number):
        my_list[bind[0]] = bind[1]
        
    for item in discount[:10]:
        if item in list(bucket.keys()):
            bucket[item] +=1
        else:
            bucket[item] = 1
    print(my_list)
    print()
    for index in range(0,len(discount)-10):
        print(bucket)
        for key in my_list.keys():
            if key not in list(bucket.keys()) or  bucket[key] != my_list[key]:
                state = False
                break
        if state:
            total+=1
        else:
            state = True
        
        if discount[index+10] not in list(bucket.keys()):
            bucket[discount[index+10]] = 1
            bucket[discount[index]]-=1
            if bucket[discount[index]] == 0:
                del bucket[discount[index]]
        else:
            bucket[discount[index+10]] += 1
            bucket[discount[index]]-=1
            if bucket[discount[index]] == 0:
                del bucket[discount[index]]
            
    return total
solution(a,b,c)   

{'banana': 3, 'apple': 2, 'rice': 2, 'pork': 2, 'pot': 1}

{'chicken': 1, 'apple': 3, 'banana': 2, 'rice': 2, 'pork': 2}
{'apple': 3, 'banana': 2, 'rice': 2, 'pork': 2, 'pot': 1}
{'apple': 2, 'banana': 3, 'rice': 2, 'pork': 2, 'pot': 1}
{'apple': 2, 'banana': 3, 'rice': 2, 'pork': 2, 'pot': 1}


2